<a href="https://colab.research.google.com/github/Osw1997/An-Introduction-to-Statistical-Learning/blob/main/Scripts/Classification/applied_exercises_ch4_classification_Chapter_3_Linear_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 4 - Classification


# Applied exercises
---
---
---
## Exercise 13
## Involved dataset: Weekly
13. This question should be answered using the Weekly data set, which is part of the ISLR2 package. This data is similar in nature to the Smarket data from this chapter’s lab, except that it contains 1, 089 weekly returns for 21 years, from the beginning of 1990 to the end of 2010.

In [1]:
!pip install --upgrade scipy
!pip install --upgrade plotly

     |████████████████████████████████| 38.1 MB 32.1 MB/s 
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.
     |████████████████████████████████| 27.7 MB 60.5 MB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 5.5.0
    Uninstalling plotly-5.5.0:
      Successfully uninstalled plotly-5.5.0


In [2]:
import pandas as pd
import os
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import t

In [3]:
import scipy
scipy.__version__

'1.7.3'

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
weekly = pd.read_csv('drive/MyDrive/itsl_datasets/Weekly.csv')
weekly.columns

Index(['Year', 'Lag1', 'Lag2', 'Lag3', 'Lag4', 'Lag5', 'Volume', 'Today',
       'Direction'],
      dtype='object')

In [7]:
weekly.describe()

,Year,Lag1,Lag2,Lag3,Lag4,Lag5,Volume,Today
count,1089.000000,1089.000000,1089.000000,1089.000000,1089.000000,1089.000000,1089.000000,1089.000000
mean,2000.048669,0.150585,0.151079,0.147205,0.145818,0.139893,1.574618,0.149899
std,6.033182,2.357013,2.357254,2.360502,2.360279,2.361285,1.686636,2.356927
min,1990.000000,-18.195000,-18.195000,-18.195000,-18.195000,-18.195000,0.087465,-18.195000
25%,1995.000000,-1.154000,-1.154000,-1.158000,-1.158000,-1.166000,0.332022,-1.154000
50%,2000.000000,0.241000,0.241000,0.241000,0.238000,0.234000,1.002680,0.241000
75%,2005.000000,1.405000,1.409000,1.409000,1.409000,1.405000,2.053727,1.405000
max,2010.000000,12.026000,12.026000,12.026000,12.026000,12.026000,9.328214,12.026000


In [8]:
weekly.head()

,Year,Lag1,Lag2,Lag3,Lag4,Lag5,Volume,Today,Direction
0,1990,0.816,1.572,-3.936,-0.229,-3.484,0.154976,-0.270,Down
1,1990,-0.270,0.816,1.572,-3.936,-0.229,0.148574,-2.576,Down
2,1990,-2.576,-0.270,0.816,1.572,-3.936,0.159837,3.514,Up
3,1990,3.514,-2.576,-0.270,0.816,1.572,0.161630,0.712,Up
4,1990,0.712,3.514,-2.576,-0.270,0.816,0.153728,1.178,Up


In [30]:
weekly['Direction'].unique() # Down, Up
# One one encoding (ohe)
weekly_ohe = weekly.copy()
weekly_ohe['Direction'] = pd.get_dummies(weekly_ohe['Direction']).drop('Down', axis=1)
weekly_ohe.describe()

,Year,Lag1,Lag2,Lag3,Lag4,Lag5,Volume,Today,Direction
count,1089.000000,1089.000000,1089.000000,1089.000000,1089.000000,1089.000000,1089.000000,1089.000000,1089.000000
mean,2000.048669,0.150585,0.151079,0.147205,0.145818,0.139893,1.574618,0.149899,0.555556
std,6.033182,2.357013,2.357254,2.360502,2.360279,2.361285,1.686636,2.356927,0.497132
min,1990.000000,-18.195000,-18.195000,-18.195000,-18.195000,-18.195000,0.087465,-18.195000,0.000000
25%,1995.000000,-1.154000,-1.154000,-1.158000,-1.158000,-1.166000,0.332022,-1.154000,0.000000
50%,2000.000000,0.241000,0.241000,0.241000,0.238000,0.234000,1.002680,0.241000,1.000000
75%,2005.000000,1.405000,1.409000,1.409000,1.409000,1.405000,2.053727,1.405000,1.000000
max,2010.000000,12.026000,12.026000,12.026000,12.026000,12.026000,9.328214,12.026000,1.000000


In [31]:
weekly_ohe.head()

,Year,Lag1,Lag2,Lag3,Lag4,Lag5,Volume,Today,Direction
0,1990,0.816,1.572,-3.936,-0.229,-3.484,0.154976,-0.270,0
1,1990,-0.270,0.816,1.572,-3.936,-0.229,0.148574,-2.576,0
2,1990,-2.576,-0.270,0.816,1.572,-3.936,0.159837,3.514,1
3,1990,3.514,-2.576,-0.270,0.816,1.572,0.161630,0.712,1
4,1990,0.712,3.514,-2.576,-0.270,0.816,0.153728,1.178,1


In [34]:

# (a) Produce some numerical and graphical summaries of the Weekly data. Do there appear to be any patterns?
import plotly.express as px
numeric_columns = ['Year', 'Lag1', 'Lag2', 'Lag3', 'Lag4', 'Lag5', 'Volume'
                   , 'Today'
                   , 'Direction'
                   ]

fig = px.scatter_matrix(weekly_ohe[numeric_columns])
fig.update_traces(diagonal_visible=False)
fig.show()



import plotly.express as px
import numpy as np
corr_matrix = np.corrcoef(weekly_ohe[numeric_columns], rowvar=False)

# corr_matrix

fig = px.imshow(corr_matrix
                , text_auto=True
                , labels=dict(x="Columns", y="Columns")
                , x=numeric_columns
                , y=numeric_columns
                , aspect="auto"
                , color_continuous_scale="BuPu"
                )
fig.update_xaxes(side="top")
fig.show()
# It seems that exists a correlation between the column "Volume" and "Year", 
# and "Direction" and "Today".


In [61]:
# (b) Use the full data set to perform a logistic regression with Direction as the response and the five lag variables
#  plus Volume as predictors. Use the summary function to print the results. Do any of the predictors appear to be statistically significant? 
#  If so, which ones?
# from sklearn.linear_model import LogisticRegression
# lr = LogisticRegression()
# response = ['Direction']
# predictors = ['Lag1', 'Lag2', 'Lag3', 'Lag4', 'Lag5', 'Volume']
# lr.fit(X=weekly_ohe[predictors], y=weekly[response])

import statsmodels.formula.api as smf
log_reg = smf.logit(formula="Direction ~ Lag1 + Lag2 + Lag3 + Lag4 + Lag5 + Volume", data=weekly_ohe).fit()
log_reg.summary2()
# # Predictors statistically significant:
# Intersect & Lag2

# (c) Compute the confusion matrix and overall fraction of correct predictions. 
# Explain what the confusion matrix is telling you about the types of mistakes made by logistic regression.
from sklearn.metrics import confusion_matrix
threshold = 0.5
y_pred = log_reg.predict() > threshold
conf_mat = confusion_matrix(weekly_ohe['Direction'], y_pred)
print(conf_mat)
print(f"Overall fraction of correct predictions: {conf_mat[0][0]} + {conf_mat[1][1]} / {len(y_pred)} =  {(conf_mat[0][0] + conf_mat[1][1])/len(y_pred)}")
# We can see that our predictor is better that guessing the outcome. (0.56 > 0.5)


Optimization terminated successfully.
         Current function value: 0.682441
         Iterations 4
[[ 54 430]
 [ 48 557]]
Overall fraction of correct predictions: 54 + 557 / 1089 =  0.5610651974288338


In [118]:
# (d) Now fit the logistic regression model using a training data period from 1990 to 2008, with Lag2 as the only predictor.
weekly_ohe_90_08 = weekly_ohe[(weekly_ohe['Year'] >= 1990) & (weekly_ohe['Year'] <= 2008)]
log_reg_v2 = smf.logit(formula="Direction ~ Lag2", data=weekly_ohe_90_08).fit()
log_reg_v2.summary2()
#  Compute the confusion matrix and the overall fraction of correct predictions for the held out data (that is, the data from 2009 and 2010).
weekly_ohe_08 = weekly_ohe[weekly_ohe['Year'] > 2008]
threshold = 0.5
y_pred = log_reg_v2.predict(weekly_ohe_08) > threshold
conf_mat = confusion_matrix(weekly_ohe_08['Direction'], y_pred)
print(conf_mat)
print(f"Overall fraction of correct predictions: {conf_mat[0][0]} + {conf_mat[1][1]} / {len(y_pred)} =  {(conf_mat[0][0] + conf_mat[1][1])/len(y_pred)}")


from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
response = ['Direction']
predictors = ['Lag2']
lr.fit(X=weekly_ohe_90_08[predictors], y=weekly_ohe_90_08[response].values.ravel())
y_pr = lr.predict(X=weekly_ohe_08[predictors])
conf_mat = confusion_matrix(weekly_ohe_08['Direction'], y_pr)
print(conf_mat)
print(f"Overall fraction of correct predictions: {conf_mat[0][0]} + {conf_mat[1][1]} / {len(y_pred)} =  {(conf_mat[0][0] + conf_mat[1][1])/len(y_pred)}")
print(f"Score: {lr.score(X=weekly_ohe_08[predictors], y=weekly_ohe_08[response])}")



Optimization terminated successfully.
         Current function value: 0.685555
         Iterations 4
[[ 9 34]
 [ 5 56]]
Overall fraction of correct predictions: 9 + 56 / 104 =  0.625
[[ 9 34]
 [ 5 56]]
Overall fraction of correct predictions: 9 + 56 / 104 =  0.625
Score: 0.625


In [103]:
weekly_ohe_90_08['Lag2'].shape

(985,)

In [128]:
# (e) Repeat (d) using LDA. LINEAR DISCRIMINANT ANALYSIS
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
print("=" * 30 + ' >>> Linear Discriminant Analysis (LDA) <<<' + "=" * 30)

lda = LinearDiscriminantAnalysis()
response = ['Direction']
predictors = ['Lag2']
lda_trained = lda.fit(X=weekly_ohe_90_08[predictors], y=weekly_ohe_90_08[response].values.ravel())
y_pred = lda_trained.predict(X=weekly_ohe_08[predictors])
conf_matrix = confusion_matrix(weekly_ohe_08[response], y_pred)
print(f"Overall fraction of correct predictions: {conf_mat[0][0]} + {conf_mat[1][1]} / {len(y_pred)} =  {(conf_mat[0][0] + conf_mat[1][1])/len(y_pred)}")
print(f"Score: {lr.score(X=weekly_ohe_08[predictors], y=weekly_ohe_08[response])}")




# (f) Repeat (d) using QDA. QUADRATIC DISCRIMINANT ANALYSIS
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
print("=" * 30 + ' >>> Linear Discriminant Analysis (LDA) <<<' + "=" * 30)

model = QuadraticDiscriminantAnalysis()
response = ['Direction']
predictors = ['Lag2']
model_trained = model.fit(X=weekly_ohe_90_08[predictors], y=weekly_ohe_90_08[response].values.ravel())
y_pred = model_trained.predict(X=weekly_ohe_08[predictors])
conf_matrix = confusion_matrix(weekly_ohe_08[response], y_pred)
print(f"Overall fraction of correct predictions: {conf_mat[0][0]} + {conf_mat[1][1]} / {len(y_pred)} =  {(conf_mat[0][0] + conf_mat[1][1])/len(y_pred)}")
print(f"Score: {lr.score(X=weekly_ohe_08[predictors], y=weekly_ohe_08[response])}")

# (g) Repeat (d) using KNN with K = 1.
from sklearn.neighbors import KNeighborsClassifier
print("=" * 30 + ' >>> K-Neighbors (N=1) <<<' + "=" * 30)

model = KNeighborsClassifier(n_neighbors=1)
response = ['Direction']
predictors = ['Lag2']
model_trained = model.fit(X=weekly_ohe_90_08[predictors], y=weekly_ohe_90_08[response].values.ravel())
y_pred = model_trained.predict(X=weekly_ohe_08[predictors])
conf_matrix = confusion_matrix(weekly_ohe_08[response], y_pred)
print(f"Overall fraction of correct predictions: {conf_mat[0][0]} + {conf_mat[1][1]} / {len(y_pred)} =  {(conf_mat[0][0] + conf_mat[1][1])/len(y_pred)}")
print(f"Score: {lr.score(X=weekly_ohe_08[predictors], y=weekly_ohe_08[response])}")

# (h) Repeat (d) using naive Bayes.
from sklearn.naive_bayes import GaussianNB
print("=" * 30 + ' >>> Naive Bayes <<<' + "=" * 30)

model = GaussianNB()
response = ['Direction']
predictors = ['Lag2']
model_trained = model.fit(X=weekly_ohe_90_08[predictors], y=weekly_ohe_90_08[response].values.ravel())
y_pred = model_trained.predict(X=weekly_ohe_08[predictors])
conf_matrix = confusion_matrix(weekly_ohe_08[response], y_pred)
print(f"Overall fraction of correct predictions: {conf_mat[0][0]} + {conf_mat[1][1]} / {len(y_pred)} =  {(conf_mat[0][0] + conf_mat[1][1])/len(y_pred)}")
print(f"Score: {lr.score(X=weekly_ohe_08[predictors], y=weekly_ohe_08[response])}")

============================== >>> Linear Discriminant Analysis (LDA) <<<==============================
Overall fraction of correct predictions: 9 + 56 / 104 =  0.625
Score: 0.625
============================== >>> Linear Discriminant Analysis (LDA) <<<==============================
Overall fraction of correct predictions: 9 + 56 / 104 =  0.625
Score: 0.625
============================== >>> K-Neighbors (N=1) <<<==============================
Overall fraction of correct predictions: 9 + 56 / 104 =  0.625
Score: 0.625
============================== >>> Naive Bayes <<<==============================
Overall fraction of correct predictions: 9 + 56 / 104 =  0.625
Score: 0.625


## (i) Which of these methods appears to provide the best results on this data?
### **A = All of them has the same score (0.625) but it's better to chose the method that implies less computational cost.**

### (j) Experiment with different combinations of predictors, including possible transformations and interactions, for each of the methods. Report the variables, method, and associated confusion matrix that appears to provide the best results on the held out data. Note that you should also experiment with values for K in the KNN classifier.

array([1, 1, 1, ..., 1, 1, 1], dtype=uint8)